In [1]:
# This code is written at BigVision LLC. It is based on the OpenCV project. It is subject to the license terms in the LICENSE file found in this distribution and at http://opencv.org/license.html

# Usage example:  python3 object_detection_yolo.py --video=run.mp4
#                 python3 object_detection_yolo.py --image=bird.jpg

import cv2 as cv
import argparse
import sys
import numpy as np
import os.path
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Initialize the parameters
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4  #Non-maximum suppression threshold

inpWidth = 416  #608     #Width of network's input image
inpHeight = 416 #608     #Height of network's input image

parser = argparse.ArgumentParser(description='Object Detection using YOLO in OPENCV')
parser.add_argument('--image', help='Path to image file.')
parser.add_argument('--video', help='Path to video file.')

class abc():
    def __init__(self, name):
        self.name = name

# Load names of classes
classesFile = "classes.names";

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.

modelConfiguration = "darknet-yolov3.cfg";
modelWeights = "lapi.weights";

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Draw the predicted bounding box
def drawPred(classId, conf, left, top, right, bottom):
    # Draw a bounding box.
    #    cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    crop = cv.rectangle(frame, (left, top), (right, bottom), (0, 0, 0), 3)

    #label = '%.2f' % conf

    # Get the label for the class name and its confidence
    #if classes:
    #    assert(classId < len(classes))
    #    label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    #labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    #top = max(top, labelSize[1])
    #crop = cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (0, 255, 0), cv.FILLED)
    #cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine),    (255, 255, 255), cv.FILLED)
    #cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)
    cv.imshow("Detection",crop)
    #v.imshow("Car plate",crop[top:bottom,left:right])
    cv.imwrite('test100.jpg', crop[top-1:bottom+1,left+1:right-1])
    
    image = cv.imread('test100.jpg')
    text = pytesseract.image_to_string(image, config='--oem 3 --psm 12')
    print('Car plate: ' + text)

# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        print("out.shape : ", out.shape)
        for detection in out:
            #if detection[4]>0.001:
            scores = detection[5:]
            classId = np.argmax(scores)
            #if scores[classId]>confThreshold:
            confidence = scores[classId]
            if detection[4]>confThreshold:
                print(detection[4], " - ", scores[classId], " - th : ", confThreshold)
                print(detection)
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)

# outputFile = "yolo_out_py.avi"

#     # Webcam input
cap = cv.VideoCapture(0)

# Get the video writer initialized to save the output video
# vid_writer = cv.VideoWriter(outputFile, cv.VideoWriter_fourcc('M','J','P','G'), 60, (round(cap.get(cv.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv.CAP_PROP_FRAME_HEIGHT))))

while (True):

    # get frame from the video
    hasFrame, frame = cap.read()
    
    if hasFrame == True:
        cv.imshow('Deep learning object detection in OpenCV', frame)

        # Create a 4D blob from a frame.
        blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(getOutputsNames(net))

    # Remove the bounding boxes with low confidence
        postprocess(frame, outs)
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            
    else:
        break

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

Car plate: mal

EBA a0

out.shape :  (507, 6)
0.99739  -  0.9967062  - th :  0.5
[0.5742616  0.5133659  0.32415166 0.13436528 0.99739    0.9967062 ]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: 
out.shape :  (507, 6)
0.9973417  -  0.9965934  - th :  0.5
[0.49837065 0.52117693 0.32670456 0.13347937 0.9973417  0.9965934 ]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: SBA 12344

out.shape :  (507, 6)
0.9139478  -  0.9129152  - th :  0.5
[0.5479345  0.5303134  0.2885808  0.12604843 0.9139478  0.9129152 ]
0.63886184  -  0.0  - th :  0.5
[0.5470005  0.547392   0.29630336 0.13196306 0.63886184 0.        ]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: SBA 1234)

out.shape :  (507, 6)
0.985595  -  0.9846107  - th :  0.5
[0.5198246  0.51669514 0.2843661  0.1409372  0.985595   0.9846107 ]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: 
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
o

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
0.96399885  -  0.9593993  - th :  0.5
[0.39445785 0.20509392 0.2807082  0.16721743 0.96399885 0.9593993 ]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: a

—=

out.shape :  (507, 6)
0.9045913  -  0.89903164  - th :  0.5
[0.39477175 0.19994906 0.28877395 0.1595287  0.9045913  0.89903164]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: PR

out.shape :  (507, 6)
0.92561305  -  0.9217104  - th :  0.5
[0.3988733  0.26701105 0.31125796 0.14184478 0.92561305 0.9217104 ]
out.shape :  (2028, 6)
out.shape :  (8112, 6)
Car plate: LSE

out.shape :  (507,

...

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out.shape :  (8112, 6)
out.shape :  (507, 6)
out.shape :  (2028, 6)
out

KeyboardInterrupt: 

In [1]:
from PIL import Image
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #After installation, find tesseract.exe on your directoory

test_set = [1,4,5,6,7,10,14,17,18,20,21,22,24,26,27,28,29,30]
answer = ['PEN 15','WYK 7460', 'HP 2368', 'QAA1989C','BJD168','SAA 853 J', 'SUKOM 186'
         , 'HWC 2012', 'VAH 3138', 'VW 4934' , 'VEJ 6036', 'WB 387 D', 'ALD7777', 'VCE 4522'
         , 'VCP 9068', 'VCQ 9868', 'WD 1253 B', 'JKG 7613']
i = 0
for test in test_set:
    image = Image.open("C:/Users/TRAINING/Downloads/character detection-20200902T041452Z-001/character detection/car1.png") # change to folder directory
    text = pytesseract.image_to_string(image, config='--oem 3 --psm 12')
    print('Car plate: ' + text)
    i = i+1

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee

Car plate: fa bee



In [2]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.5-py2.py3-none-any.whl size=13410 sha256=c86bc7d87ef060c29536841c70c253be914ea637ebd561410025b44d95dae6dd
  Stored in directory: c:\users\training\appdata\local\pip\cache\wheels\7c\b0\92\034f16549e825401698ccb3f74322f39e770b6aad121188af3
Successfully built pytesseract
